In [1]:
%load_ext lab_black

In [2]:
import requests, json, random
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
from config import weather_api_key
from config import g_key
from config import mapbox_token
from citipy import citipy

# from sklearn.linear_model import LinearRegression
from scipy import stats
from datetime import date

In [3]:
px.set_mapbox_access_token(mapbox_token)

In [4]:
# Build list of random coordinates

coordinate_list = []
counter = 0
number_of_points = 500

while counter < number_of_points:
    lat = round(random.uniform(-90, 90), 6)
    lon = round(random.uniform(-180, 180), 6)
    counter += 1
    coordinate_list.append((lat, lon))

# Assign nearest city to given set of coordinates
cities = []
city_count = 0

for coords in coordinate_list:
    city = citipy.nearest_city(coords[0], coords[1])
    if city.city_name not in cities:
        cities.append(city.city_name)
        city_count += 1

In [5]:
# Obtain weather data for cities in the list

cities_results = []
count = 0
found_count = 0
not_found_count = 0

print("Beginning data retrieval:\n")
print(50 * "-")

for citi in cities:
    endpoint = "http://api.openweathermap.org/data/2.5/weather"
    payload = {"q": citi, "appid": weather_api_key, "units": "imperial"}

    r = requests.get(url=endpoint, params=payload)

    count += 1

    if r.status_code == 200:
        data = r.json()
        cities_results.append(data)
        print("Processing record: ", count, " | ", citi)
        found_count += 1
    else:
        print("Processing record: ", count, " | ", citi, " *Not Found*")
        not_found_count += 1

Beginning data retrieval:

--------------------------------------------------
Processing record:  1  |  thompson
Processing record:  2  |  kirakira
Processing record:  3  |  illoqqortoormiut  *Not Found*
Processing record:  4  |  mataura
Processing record:  5  |  ostrovnoy
Processing record:  6  |  albany
Processing record:  7  |  la ronge
Processing record:  8  |  sozimskiy
Processing record:  9  |  kahului
Processing record:  10  |  tuktoyaktuk
Processing record:  11  |  cape town
Processing record:  12  |  khatanga
Processing record:  13  |  barrow
Processing record:  14  |  taburi  *Not Found*
Processing record:  15  |  marsh harbour
Processing record:  16  |  urdzhar  *Not Found*
Processing record:  17  |  port elizabeth
Processing record:  18  |  katsuura
Processing record:  19  |  hermanus
Processing record:  20  |  labuhan
Processing record:  21  |  cayenne
Processing record:  22  |  gaoual
Processing record:  23  |  anage
Processing record:  24  |  hilo
Processing record:  25 

In [6]:
r.status_code

200

In [7]:
r.url

'http://api.openweathermap.org/data/2.5/weather?q=cockburn+town&appid=37a1498de740e2228eee5f21decc9230&units=imperial'

In [8]:
found_count

246

In [9]:
not_found_count

27

In [11]:
cities_results[0]

{'coord': {'lon': -97.86, 'lat': 55.74},
 'weather': [{'id': 804,
   'main': 'Clouds',
   'description': 'overcast clouds',
   'icon': '04n'}],
 'base': 'stations',
 'main': {'temp': 48.2,
  'feels_like': 43.65,
  'temp_min': 48.2,
  'temp_max': 48.2,
  'pressure': 994,
  'humidity': 87},
 'visibility': 10000,
 'wind': {'speed': 5.82, 'deg': 130},
 'clouds': {'all': 90},
 'dt': 1601013540,
 'sys': {'type': 1,
  'id': 935,
  'country': 'CA',
  'sunrise': 1601036619,
  'sunset': 1601079727},
 'timezone': -18000,
 'id': 6165406,
 'name': 'Thompson',
 'cod': 200}

In [13]:
city_data = []

for city in cities_results:
    city_data.append(
        {
            "City": city["name"],
            "Country": city["sys"]["country"],
            "Lat": city["coord"]["lat"],
            "Long": city["coord"]["lon"],
            "Temp_max": city["main"]["temp_max"],
            "Humidity": city["main"]["humidity"],
            "Cloudiness": city["clouds"]["all"],
            "Wind_Speed": city["wind"]["speed"],
        }
    )

city_weather = pd.DataFrame(city_data)
city_weather
# city_table.to_csv("data/city_weather_data.csv", index=False)

,City,Country,Lat,Long,Temp_max,Humidity,Cloudiness,Wind_Speed
0,Thompson,CA,55.74,-97.86,48.20,87,90,5.82
1,Kirakira,SB,-10.45,161.92,81.72,78,94,8.01
2,Mataura,NZ,-46.19,168.86,51.01,72,88,3.00
3,Ostrovnoy,RU,68.05,39.51,54.86,83,98,20.56
4,Albany,US,42.60,-73.97,62.01,75,70,1.01
...,...,...,...,...,...,...,...,...
241,Semey,KZ,50.41,80.23,46.40,87,90,20.13
242,Winslow,US,35.02,-110.70,66.20,24,1,8.05
243,Kruisfontein,ZA,-34.00,24.73,61.00,83,9,1.01
244,Pacific Grove,US,36.62,-121.92,63.00,86,0,7.00


In [15]:
city_weather.loc[
    (city_weather["Cloudiness"] == 0)
    & (city_weather["Wind_Speed"] < 10)
    & (city_weather["Temp_max"] > 70)
    & (city_weather["Temp_max"] < 80)
]

,City,Country,Lat,Long,Temp_max,Humidity,Cloudiness,Wind_Speed
23,Sabha,LY,27.04,14.43,79.70,29,0,2.66
112,Aksu,CN,41.12,80.26,77.76,18,0,4.52
160,Yulara,AU,-25.24,130.99,78.80,13,0,6.93
233,Ondjiva,AO,-17.07,15.73,70.52,23,0,2.89
